# **Fake News Detection using Machine Learning**

## **Importing Libraries**

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

## **Data Loading and Preprocessing**

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
true = pd.read_csv('/content/drive/MyDrive/Data Sets/Fake News Detection/True.csv')
fake = pd.read_csv('/content/drive/MyDrive/Data Sets/Fake News Detection/Fake.csv')

In [4]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [6]:
true['label'] = 1
fake['label'] = 0

In [7]:
df_news = pd.concat([true, fake], axis = 0)
df_news

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0


In [8]:
df_news.isnull().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

In [9]:
df_news.drop(['title', 'subject', 'date'], axis=1, inplace=True)
df_news = df_news.sample(frac=1)  # Reshuffle Dataset
df_news.reset_index(inplace=True) # Reset indexes

df_news.head()

,index,text,label
0,14178,The unborn person is how Clinton describes t...,0
1,5490,WASHINGTON (Reuters) - Washington State Attorn...,1
2,7904,MOSCOW (Reuters) - Kremlin spokesman Dmitry Pe...,1
3,19395,SYDNEY (Reuters) - Australian police charged a...,1
4,20559,Culture rot The health professionals who will ...,0


In [10]:
df_news.drop(['index'], axis=1, inplace=True)
df_news.head()

,text,label
0,The unborn person is how Clinton describes t...,0
1,WASHINGTON (Reuters) - Washington State Attorn...,1
2,MOSCOW (Reuters) - Kremlin spokesman Dmitry Pe...,1
3,SYDNEY (Reuters) - Australian police charged a...,1
4,Culture rot The health professionals who will ...,0


## **Text Processing**

In [11]:
import re

def process_text(text):
  """
  This function takes a string as input and performs the following operations:
  1. Converts the text to lowercase
  2. Removes URLs
  3. Removes HTML tags
  4. Removes punctuation
  5. Removes digits
  6. Removes newline characters

  Args:
      text: The string to be processed.

  Returns:
      The processed string.
  """

  # Convert the text to lowercase
  text = text.lower()

  # Remove URLs
  text = re.sub(r"https?://\S+/www\.\S+", "", text)

  # Remove HTML tags
  text = re.sub(r"<.*?>", "", text)

  # Remove punctuation
  text = re.sub(r"[^\w\s]", "", text)

  # Remove digits
  text = re.sub(r"\d", "", text)

  # Remove newline characters
  text = re.sub(r"\n", "", text)

  return text

In [12]:
df_news['text'] = df_news['text'].apply(process_text)
df_news

,text,label
0,the unborn person is how clinton describes t...,0
1,washington reuters washington state attorney ...,1
2,moscow reuters kremlin spokesman dmitry pesko...,1
3,sydney reuters australian police charged a ye...,1
4,culture rot the health professionals who will ...,0
...,...,...
44893,toledo ohio winston salem nc reuters us repub...,1
44894,the funniest part of the video comes when msnb...,0
44895,chicago reuters a fiscal budget unveiled on ...,1
44896,this is possibly the most disturbing video we ...,0


## **Train-Test Split**

In [13]:
x = df_news['text']
y = df_news['label']

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

print(x_train.shape)
print(x_test.shape)

(31428,)
(13470,)


## **Text Vectorization**

In [15]:
vectorization = TfidfVectorizer()

In [16]:
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

## **Model Training and Evaluation**

In [17]:
lr = LogisticRegression()
lr.fit(xv_train, y_train)

LogisticRegression()

In [18]:
y_pred_lr = lr.predict(xv_test)

In [19]:
print("Logistic Regression Score:", lr.score(xv_test, y_test))

Logistic Regression Score: 0.9883444691907943


In [20]:
print(classification_report(y_test, y_pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      7083
           1       0.99      0.99      0.99      6387

    accuracy                           0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470



In [21]:
dtc = DecisionTreeClassifier()
dtc.fit(xv_train, y_train)
y_pred_dtc = dtc.predict(xv_test)

In [22]:
print("Decision Tree Classifier Score:", dtc.score(xv_test, y_test))

Decision Tree Classifier Score: 0.995916852264291


In [23]:
print(classification_report(y_test, y_pred_dtc))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7083
           1       1.00      1.00      1.00      6387

    accuracy                           1.00     13470
   macro avg       1.00      1.00      1.00     13470
weighted avg       1.00      1.00      1.00     13470



In [24]:
rfc = RandomForestClassifier()
rfc.fit(xv_train, y_train)
y_pred_rfc = rfc.predict(xv_test)

In [25]:
print("Random Forest Classifier Score:", rfc.score(xv_test, y_test))

Random Forest Classifier Score: 0.9848552338530067


In [26]:
print(classification_report(y_test, y_pred_rfc))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      7083
           1       0.98      0.99      0.98      6387

    accuracy                           0.98     13470
   macro avg       0.98      0.98      0.98     13470
weighted avg       0.98      0.98      0.98     13470



In [27]:
gbc = GradientBoostingClassifier()
gbc.fit(xv_train, y_train)
pred_gbc = gbc.predict(xv_test)

In [28]:
print("Gradient Boosting Classifier Score:", gbc.score(xv_test, y_test))
print(classification_report(y_test, pred_gbc))

Gradient Boosting Classifier Score: 0.9953229398663697
              precision    recall  f1-score   support

           0       1.00      0.99      1.00      7083
           1       0.99      1.00      1.00      6387

    accuracy                           1.00     13470
   macro avg       1.00      1.00      1.00     13470
weighted avg       1.00      1.00      1.00     13470



## **Manual Testing Function**

In [29]:
def output_label(n):
  if n == 0:
    return "It is a Fake News"
  elif n == 1:
    return "It is a Genuine News"
  else:
    return "Error: Unexpected prediction value"

In [36]:
def manual_testing(news):
  """
  This function performs manual testing on a news article by converting it to lowercase, removing special characters and URLs, and then applying a custom word optimization function.

  Args:
      news: The news article string to be processed.

  Returns:
      The processed news article string.
  """
  testing_news = {"text": [news]} # Corrected syntax for defining dictionary

  # Convert the text to lowercase
  new_def_test = pd.DataFrame(testing_news)
  new_def_test["text"] = new_def_test["text"].apply(process_text)  # Assuming 'wordopt' is a custom function

  # Vectorize the text data (using an assumed vectorizer object)
  new_x_test = new_def_test["text"]
  new_xv_test = vectorization.transform(new_x_test)

  # Predict the class labels using multiple machine learning models
  pred_lr = lr.predict(new_xv_test)  # Logistic Regression prediction
  # pred_dtc = dtc.predict(new_xv_test) # Commented out decision tree classifier
  pred_gbc = gbc.predict(new_xv_test)  # Gradient Boosting Classifier prediction
  pred_rfc = rfc.predict(new_xv_test)  # Random Forest Classifier prediction

  # Combine the predictions and format the output string
  return "\n\nLR Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(
      output_label(pred_lr[0]), output_label(pred_gbc[0]), output_label(pred_rfc[0]))


In [43]:
news_article = str(input())

In an unexpected turn of events, astronomers from around the globe have reported unusual activity in the cosmos that could indicate a looming alien invasion. The discovery, made during routine observations of distant galaxies, has sent shockwaves through the scientific community and sparked widespread concern among the general public.  The first signs of trouble emerged late last week when astronomers at the renowned Kepler Space Observatory detected a series of peculiar radio signals emanating from a remote corner of the universe. Initial analysis of the signals revealed patterns that defied conventional explanations, leading experts to speculate that they could be of extraterrestrial origin.  Dr. Sarah Johnson, lead researcher at the Kepler Observatory, described the discovery as "both thrilling and terrifying." "We've been searching for signs of intelligent life beyond our solar system for decades, but this is the first time we've encountered anything quite like this," she said in a

In [44]:
manual_testing(news_article)

'\n\nLR Prediction: It is a Fake News \nGBC Prediction: It is a Fake News \nRFC Prediction: It is a Fake News'